In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.train import main
from src.utils import prepare_data

data_filepath = "../../data/processed.csv"

In [3]:
trn_df, val_df = prepare_data(data_filepath)

In [5]:
main(trn_df, val_df)

06/04/2022 22:50:59 - WARNING - src.train -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/04/2022 22:51:01 - INFO - src.train -   Training/evaluation parameters <src.train.Args object at 0x7fd74adedff0>
06/04/2022 22:51:01 - INFO - src.train -   Creating features from dataset file at cached
06/04/2022 22:51:18 - INFO - src.train -   Saving features into cached file cached/gpt2_cached_lm_512
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
06/04/2022 22:51:18 - INFO - src.train -   ***** Running training *****
06/04/2022 22:51:18 - INFO - src.train -     Num examples = 28874
06/04/2022 22:51:18 - INFO - src.train -     Num Epochs = 3
06/04/2022 22:51:18 - INFO - src.train -     

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/7218 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
06/04/2022 22:57:16 - INFO - src.train -   Saving model checkpoint to output-small/checkpoint-3500
06/04/2022 22:57:17 - INFO - src.train -   Saving optimizer and scheduler states to output-small/checkpoint-3500
06/04/2022 23:03:20 - INFO - src.train -   Saving model checkpoint to output-small/checkpoint-7000
06/04/2022 23:03:21 - INFO - src.train -   Saving optimizer and scheduler states to output-small/checkpoint-7000


Iteration:   0%|          | 0/7218 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/7218 [00:00<?, ?it/s]

06/04/2022 23:21:31 - INFO - src.train -   Saving model checkpoint to output-small/checkpoint-17500
06/04/2022 23:21:32 - INFO - src.train -   Saving optimizer and scheduler states to output-small/checkpoint-17500
06/04/2022 23:27:35 - INFO - src.train -   Saving model checkpoint to output-small/checkpoint-21000
06/04/2022 23:27:36 - INFO - src.train -   Saving optimizer and scheduler states to output-small/checkpoint-21000
06/04/2022 23:28:44 - INFO - src.train -    global_step = 21654, average loss = 1.7564211775250873
06/04/2022 23:28:44 - INFO - src.train -   Saving model checkpoint to output-small
06/04/2022 23:28:45 - INFO - src.train -   Evaluate the following checkpoints: ['output-small']
06/04/2022 23:28:46 - INFO - src.train -   Creating features from dataset file at cached
06/04/2022 23:28:48 - INFO - src.train -   Saving features into cached file cached/gpt2_cached_lm_512
06/04/2022 23:28:48 - INFO - src.train -   ***** Running evaluation  *****
06/04/2022 23:28:48 - INFO -

Evaluating:   0%|          | 0/802 [00:00<?, ?it/s]

06/04/2022 23:29:09 - INFO - src.train -   ***** Eval results  *****
06/04/2022 23:29:09 - INFO - src.train -     perplexity = tensor(3.6659)


{'perplexity_': tensor(3.6659)}

In [8]:
import torch

from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output-small')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )
    
    # pretty print last ouput tokens from bot
    print("ShadowBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User: How is Shiro?


ShadowBot: She's doing better.


>> User: That's good to hear.


ShadowBot: [*charlotte will go find nix, since she's in the apts*]


>> User: So what's up with you?


ShadowBot: [shrugs* ] Just chatting


>> User: Are you sleeping okay?


ShadowBot: !!! remindme 2 days


>> User: What?


ShadowBot: ?!!remindme!! 1 day!!
